In [1]:
import pandas as pd 
import numpy as np 
import os
from os import listdir
from os.path import isfile, join

In [28]:
path = 'results/P19_48_72_images/results_filt/72hrs/aorta03'
duration = '72hr'
aorta = 'aorta03'

In [29]:
#calculate L2-distance from singletons to other cluster, other cluster sizes to other cluster and L2-distance between cells within a EdU cluster

singletonDistances = []
withinClusterDistances = []
betweenClusterDistances = []

os.chdir(path)
files = [f for f in listdir(path) if isfile(join(path, f)) if ".DS_Store" not in f if ".csv" in f if "._" not in f]
for file in files:

    ###Singletons
    ### Calculation of distances for singletons
    df_help = pd.read_csv(join(path, file))
    # get cluster sizes and how often they appear
    unique, counts = np.unique(df_help['centroid'], return_counts=True)
    #define singletons as cluster size of 1
    singletons = unique[counts == 1]
    for s in singletons:
        # Get the coordinates of the singleton
        xSingleton = df_help.loc[df_help['centroid'] == s]['X'].values[0]
        ySingleton = df_help.loc[df_help['centroid'] == s]['Y'].values[0]
        # define all other clusters as df_other
        df_other = df_help.loc[df_help['centroid'] != s]
        xother = df_other['X'].values
        yother = df_other['Y'].values
        points = np.column_stack((xother, yother))
        # Calculate L2 distances from the singleton to all other clusters
        distancesSingleton = np.linalg.norm(points - [xSingleton,ySingleton], axis=1)
        singletonDistances.append(distancesSingleton[np.argmin(distancesSingleton)])

    ### distances between all cluster 
    higher = unique[counts != 1]
    for h in higher:
        # get centroid of singletons cluster
        xvalsCluster = df_help.loc[df_help['centroid'] == h]['X'].values
        yvalsCluster = df_help.loc[df_help['centroid'] == h]['Y'].values
        pointsCluster = np.column_stack((xvalsCluster, yvalsCluster))
        df_other = df_help.loc[df_help['centroid'] != h]
        if len(df_other != 0):
            xother = df_other['X'].values
            yother = df_other['Y'].values
            pointsOther = np.column_stack((xother, yother))
            minClusterDistance = 9999999999
            for i, point in enumerate(pointsCluster):
                # Calculate L2 distances from between clusters
                distancesPoint = np.linalg.norm(pointsOther - point, axis=1)
                if minClusterDistance > np.min(distancesPoint[distancesPoint > 0]):
                    minClusterDistance = np.min(distancesPoint[distancesPoint > 0])
            betweenClusterDistances.append(minClusterDistance)

    ### Wwithin-cluster distances (only take clusters above one, because I need to calculate distance between nuclei within a cluster)
    #needed for normalization
    dfBiggerCluster = df_help[~df_help['centroid'].isin(singletons)]
    min_distances = []
    for i in np.unique(dfBiggerCluster['centroid']):
        dfCluster = df_help.loc[df_help['centroid'] == i]
        xCluster = dfCluster['X'].values
        yCluster = dfCluster['Y'].values
        points = np.column_stack((xCluster, yCluster))
        # Calculate minimum distance to other points in the same cluster
        for i, point in enumerate(points):
            distances = np.linalg.norm(points - point, axis=1)
            # Append the minimum distance that is greater than zero
            withinClusterDistances.append(np.min(distances[distances > 0]))
        


In [30]:
# normalization
normalizedSingleton = singletonDistances/np.mean(withinClusterDistances)
normalizedWithinCluster = withinClusterDistances/np.mean(withinClusterDistances)
normalizedBetweenCluster = betweenClusterDistances/np.mean(withinClusterDistances)

In [34]:
#save results
df_singleton = pd.DataFrame({'duration': duration, 'aorta': aorta, 'type': 'singleton', 'normalizedDistance': normalizedSingleton})
df_withinCluster = pd.DataFrame({'duration': duration, 'aorta': aorta, 'type': 'withinCluster', 'normalizedDistance': normalizedWithinCluster})
df_betweenCluster = pd.DataFrame({'duration': duration, 'aorta': aorta, 'type': 'betweenCluster', 'normalizedDistance': normalizedBetweenCluster})

In [35]:
dfAll = pd.concat([df_singleton, df_withinCluster, df_betweenCluster])

In [36]:
dfAll.to_csv('results/normalizedDistances_72hr_aorta03.csv', index=False)

In [48]:
### concat distances

In [50]:
df1 = pd.read_csv('results/normalizedDistances_72hr_aorta01.csv')
df2 = pd.read_csv('results/normalizedDistances_72hr_aorta02.csv')
df3 = pd.read_csv('results/normalizedDistances_72hr_aorta03.csv')

dfAllAge = pd.concat([df1, df2, df3])
dfAllAge.to_csv('results/normalizedDistances_72hr.csv', index=False)